# GolfAdvisor Topic Modeling:

<b> Description: </b> Represents the development of the topic modeling / clustering model for identifying topics from ~70k GolfAdvisor reviews

<hr>

<b> Author: </b> Derek A Maier | maierd@canisius.edu


In [9]:
# Import necessary Libraries

import os
import re
import string
import pandas as pd

# SpaCy Stop words
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# NLTK Lemmatization
from nltk.stem.porter import PorterStemmer

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Visualization
import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [11]:
# Umport DataSet
data_dir = 'data/'
data_file = os.path.join(data_dir, 'course_review_details_collections.xlsx')
target_cols = ['Course Name', 'User ID', 'Review Description']

df = pd.read_excel(data_file)
df = df[target_cols]

df.head()

,Course Name,User ID,Review Description
0,Cedar Creek Golf Course,EsyfNZTi68Ym6iNrJejQ,I usually play here early mornings (6 AM) and ...
1,Cedar Creek Golf Course,u314160524855,Fairways are crab grass. Staff rushes you on t...
2,Cedar Creek Golf Course,u5825142,The greens and fairways are faster than they e...
3,Cedar Creek Golf Course,u000004665625,The greens were kind of burnt out and the fair...
4,Cedar Creek Golf Course,Robert9245631,Course is setup well and has many challenging ...


In [55]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in STOP_WORDS] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values
        
# Cleansing the final layer and applying
def clean_data(dataframe, input_str):

    """
    Description:
    ------------
    Cleans a given input string for text preprocessing
    
    Params:
    -----------
    dataframe : (DataFrame)
        Represents a pandas DataFrame object
    input_str : (str)
        Represents a string to be cleansed

    Returns:
    -----------
    data : (str)
        A tokenized set of cleaned words from the input string

    """
    
    # Convert to list
    data = dataframe[input_str].values.tolist()
    data = [re.sub('\s+', ' ', sent) for sent in data]
    data = [re.sub("\'", "", sent) for sent in data]
    
    # Convert to lower case
    data = list(sent_to_words(data))

    '''
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    
    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    '''

    return data

In [24]:
data = clean_data(df, 'Review Description')

In [25]:
data[0]

['usually',
 'play',
 'here',
 'early',
 'mornings',
 'am',
 'and',
 'the',
 'staff',
 'and',
 'grounds',
 'crew',
 'are',
 'amazing',
 'greens',
 'could',
 'use',
 'little',
 'care',
 'but',
 'fairways',
 'are',
 'decent',
 'course',
 'layout',
 'is',
 'great',
 'and',
 'challenging',
 'for',
 'all',
 'handicaps']

In [27]:
bigram = gensim.models.Phrases(data, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data], threshold=100)  

In [29]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [31]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [32]:
data_lemmatized[0]

['usually',
 'play',
 'early',
 'morning',
 'staff',
 'amazing',
 'green',
 'use',
 'little',
 'care',
 'fairway',
 'decent',
 'course',
 'layout',
 'great',
 'challenging',
 'handicap']

In [33]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [36]:
# Human readable format of corpus (term-frequency)
tf_list = [[(id2word[id], freq) for id, freq in cp] for cp in corpus]

In [37]:
tf_list

[[('amazing', 1),
  ('care', 1),
  ('challenging', 1),
  ('course', 1),
  ('decent', 1),
  ('early', 1),
  ('fairway', 1),
  ('great', 1),
  ('green', 1),
  ('handicap', 1),
  ('layout', 1),
  ('little', 1),
  ('morning', 1),
  ('play', 1),
  ('staff', 1),
  ('use', 1),
  ('usually', 1)],
 [('course', 2),
  ('early', 1),
  ('staff', 1),
  ('box', 1),
  ('chop', 1),
  ('dirt', 1),
  ('group', 1),
  ('guy', 1),
  ('hole', 1),
  ('kind', 1),
  ('minute', 2),
  ('pace', 1),
  ('pointless', 1),
  ('poorly', 1),
  ('practice', 1),
  ('putt', 1),
  ('round', 1),
  ('run', 1),
  ('rush', 1),
  ('say', 1),
  ('show', 1),
  ('step', 1),
  ('tear', 1),
  ('tee', 3),
  ('time', 1),
  ('wait', 1),
  ('want', 1)],
 [('staff', 1),
  ('awesome', 1),
  ('complaint', 1),
  ('fast', 1),
  ('past', 1),
  ('year', 1)],
 [('fairway', 1), ('burn', 1), ('crabgrass', 1), ('rough', 1)],
 [('course', 1),
  ('hole', 1),
  ('challenge', 1),
  ('friend', 1),
  ('recommend', 1),
  ('setup', 1)],
 [('fairway', 1),
  

In [38]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [52]:
for model in lda_model.print_topics():
    print(f'Topic Number {model[0]}: \n' + '='*90 + '\n' + model[1] + '\n')

Topic Number 0: 
0.284*"course" + 0.044*"condition" + 0.043*"golf" + 0.023*"area" + 0.017*"well" + 0.016*"place" + 0.016*"enjoy" + 0.015*"excellent" + 0.015*"beautiful" + 0.015*"water"

Topic Number 1: 
0.057*"group" + 0.047*"wait" + 0.039*"player" + 0.038*"walk" + 0.033*"let" + 0.027*"leave" + 0.025*"minute" + 0.024*"offer" + 0.024*"allow" + 0.024*"help"

Topic Number 2: 
0.083*"green" + 0.082*"good" + 0.082*"hole" + 0.041*"fairway" + 0.038*"nice" + 0.032*"challenge" + 0.025*"lot" + 0.024*"layout" + 0.022*"fun" + 0.021*"little"

Topic Number 3: 
0.115*"tee" + 0.032*"hit" + 0.032*"go" + 0.024*"drive" + 0.023*"think" + 0.022*"pay" + 0.022*"people" + 0.020*"club" + 0.018*"right" + 0.018*"know"

Topic Number 4: 
0.247*"staff" + 0.157*"friendly" + 0.075*"love" + 0.043*"especially" + 0.035*"design" + 0.031*"score" + 0.029*"track" + 0.029*"level" + 0.026*"super" + 0.017*"treat"

Topic Number 5: 
0.151*"cart" + 0.045*"thing" + 0.039*"set" + 0.029*"path" + 0.029*"poor" + 0.027*"bring" + 0.021*

In [53]:
# Write the model to disk
'''
import pickle

model_dir = 'models/'
model_file = os.path.join(model_dir, 'lda_model.pickle')

with open(model_file, 'wb') as w:
    pickle.dump(lda_model, w)
'''

In [54]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.257924 -0.090926       1        1  24.874895
2     -0.219930  0.372475       2        1  22.298748
7     -0.204557 -0.284074       3        1  18.645987
3     -0.015128  0.036391       4        1  10.669697
9      0.070292  0.019521       5        1   6.312841
1      0.058886 -0.086812       6        1   5.214592
4      0.123067  0.010917       7        1   3.731915
5      0.135545  0.009393       8        1   3.581329
6      0.153918  0.006730       9        1   2.355217
8      0.155831  0.006386      10        1   2.314774, topic_info=           Term           Freq          Total Category  logprob  loglift
3        course  108106.000000  108106.000000  Default  30.0000  30.0000
13         play   63159.000000   63159.000000  Default  29.0000  29.0000
7         great   33725.000000   33725.000000  Default  28.0000  28.0000
14        staff   14098.000000   14098.000000  Default  27.0000  27.0000
8         green   28434.000000   28434.000000  Default  26.0000  26.0000
...         ...            ...            ...      ...      ...      ...
434       learn     323.767639     324.694641  Topic10  -4.6939   3.7630
1557  hopefully     299.656738     300.583679  Topic10  -4.7713   3.7628
1468    complex     296.840546     297.767548  Topic10  -4.7808   3.7627
1932      world     287.301239     288.228394  Topic10  -4.8134   3.7626
1083    comment     262.912140     263.839355  Topic10  -4.9022   3.7623

[335 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1056      8  0.997851  accuracy
646       5  0.999050       ago
324       5  0.999589     ahead
69        6  0.999490     allow
0         3  0.999604   amazing
...     ...       ...       ...
213       3  0.999614     worth
690       6  0.997844     would
278       4  0.999433      yard
891       6  0.998501   yardage
45        5  0.999951      year

[309 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 8, 4, 10, 2, 5, 6, 7, 9])

In [61]:
#mallet_path = os.path.join(data_dir, 'mallet-2.0.8/bin/mallet.bat')
#ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

In [62]:
#